<a href="https://colab.research.google.com/github/LuisaPolicarpo/Project3/blob/main/luisa_p.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [34]:
import pandas as pd
import pickle


In [35]:
# name_b = pd.read_csv('https://datasets.imdbws.com/name.basics.tsv.gz', compression = 'gzip', sep = '\t')[['nconst','primaryName','primaryProfession']]


In [36]:

# df = pd.read_csv('myfile.csv',skiprows=lambda x: x not in rownumberList)

# title_p = pd.read_csv('https://datasets.imdbws.com/title.principals.tsv.gz', compression = 'gzip', sep = '\t')[['tconst','nconst']]

In [37]:
col_list = ['nconst','primaryName','primaryProfession']


name_b = pd.read_csv('https://datasets.imdbws.com/name.basics.tsv.gz', compression = 'gzip', sep = '\t', usecols=col_list)


In [38]:
col_list2 = ['tconst','nconst']
# rownumberList = [1,2,3]
title_p = pd.read_csv('https://datasets.imdbws.com/title.principals.tsv.gz', compression = 'gzip', sep = '\t', usecols=col_list2)
# df = pd.read_csv('https://datasets.imdbws.com/title.principals.tsv.gz',skiprows=lambda x: x not in rownumberList)
# # , skiprows=lambda x: x not in rownumberList)



In [39]:
options = ['tt14022026', 'tt21386494','tt20251896','tt21963660','tt0389937','tt2061693','tt4678496','tt2899460','tt0972571','tt10940380'] 
    
# selecting rows based on condition 
rslt_df = title_p[title_p['tconst'].isin(options)] 
    
print('\nResult title_p :\n',
      rslt_df)


Result title_p :
              tconst     nconst
2789876   tt0389937  nm1043243
2789877   tt0389937  nm4188689
2789878   tt0389937  nm1097231
2789879   tt0389937  nm0379106
2789880   tt0389937  nm0676671
...             ...        ...
39292567  tt4678496  nm5942503
39292568  tt4678496  nm7321438
39292569  tt4678496  nm5949422
39292570  tt4678496  nm7321439
39292571  tt4678496  nm5854469

[89 rows x 2 columns]


In [62]:
# other = pd.DataFrame({'title_p': ['tconst', 'nconst'],
#                       'name_b': ['nconst', 'primaryName', 'primaryProfession']})

# result = pd.concat([rslt_df, name_b], axis=1, join="outer")

top_movies= pd.merge(rslt_df, name_b, how='inner', on=['nconst', 'nconst'])



In [64]:
top_movies.to_pickle('topm5.pickle')

In [42]:
# result.to_pickle('topm4.pickle')

In [65]:
top_movies5 = pd.read_pickle("/content/topm5.pickle")


In [66]:
top_movies5[['P1', 'P2', 'P3']] = top_movies5['primaryProfession'].str.split(',', expand=True)
top_movies5

,tconst,nconst,primaryName,primaryProfession,P1,P2,P3
0,tt0389937,nm1043243,Edie Caggiano,actress,actress,None,None
1,tt0389937,nm4188689,Nina Fuller,"costume_designer,casting_department,actress",costume_designer,casting_department,actress
2,tt0389937,nm1097231,Michael Tyler Henry,actor,actor,None,None
3,tt0389937,nm0379106,Carol Herman,"actress,script_department",actress,script_department,None
4,tt0389937,nm0676671,Scott Peters,"writer,producer,director",writer,producer,director
...,...,...,...,...,...,...,...
84,tt4678496,nm5942503,Glykeria Papadopoulou,actor,actor,None,None
85,tt4678496,nm7321438,Creating Memories,producer,producer,None,None
86,tt4678496,nm5949422,Giannis Papadimitriou,composer,composer,None,None
87,tt4678496,nm7321439,Konstantinos Manousaridis,"cinematographer,editor",cinematographer,editor,None


In [73]:
condition = top_movies5['P1'] == 'actor'
condition2 = top_movies5['P2'] == 'actor' 
condition3 = top_movies5['P3'] == 'actor'

# actors = name_basics2[condition | condition2 | condition3][['nconst','primaryName', 'P1','P2','P3','knownForTitles']]

actors = top_movies5[condition | condition2 | condition3][['tconst','primaryName','P1','P2','P3']]
# actors.sort_values(by=('primaryName'), ascending=False)

actors

,tconst,primaryName,P1,P2,P3
2,tt0389937,Michael Tyler Henry,actor,None,None
5,tt0389937,Don McMillan,actor,writer,producer
9,tt0972571,Luis Lake,actor,None,None
10,tt0972571,Nick Mann,actor,writer,None
16,tt10940380,Saaj Abraham,actor,None,None
18,tt10940380,Anil Bhalerao,casting_director,actor,casting_department
19,tt10940380,Jeevan Jadhav,actor,None,None
21,tt10940380,Bobby Kumar,actor,director,writer
25,tt14022026,Sanjeeb Dasgupta,actor,None,None
33,tt14022026,Ashutosh Singh,composer,music_department,actor


In [45]:
# top_movies3[['P1', 'P2', 'P3']] = top_movies3['primaryProfession'].str.split(',', expand=True)

# top_movies3

In [46]:
top_movies3.to_pickle('topm3.pickle')

In [47]:
# rownumberList = [1,2,3]
# title_p = pd.read_csv('https://datasets.imdbws.com/title.principals.tsv.gz', compression = 'gzip', sep = '\t', usecols=col_list2)

In [48]:
# name_b.to_pickle("zz-name.basics.pickle")
# title_p.to_pickle("zz-title.principals.pickle")

In [49]:
# name_basics= name_b.drop(['birthYear', 'deathYear','knownForTitles'],axis=1)

# name_basics.head()

In [50]:
# title_pri= title_p.drop(['category','job','characters'],axis=1)

# title_pri.head()

In [51]:
# top_movies= pd.merge(title_p, name_b, how='inner', on=['nconst', 'nconst'])
# # top_movies.to_pickle('topm.pickle')

# top_movies = pd.concat([title_p, name_b], axis=1, join="inner")


In [52]:
# top_movies[['P1', 'P2', 'P3']] = top_movies['primaryProfession'].str.split(',', expand=True)

# top_movies

In [53]:
# top_movies2 = pd.read_pickle("/content/topm.pickle")
# top_movies2 = top_movies2.drop(['ordering'],axis=1)
# top_movies2 = top_movies2.drop_duplicates()

In [54]:
# top_movies2.head(30)

In [55]:
#top_movies2 duplicates and ordering

# top_movies2 = top_movies2.drop(['ordering'],axis=1)

# top_movies2 = top_movies2.drop_duplicates()



In [56]:
# top_movies2[['P1', 'P2', 'P3']] = top_movies2['primaryProfession'].str.split(',', expand=True)

# top_movies2

In [57]:
# top_movies2['P1'] = top_movies2['primaryProfession'].str.split(',')[0]

In [58]:
# top_movies2.to_pickle('topm2.pickle')

In [59]:
# top_movies3 = pd.read_pickle("/content/topm.pickle")
# top_movies3